# Weekly speed averages

In [1]:
import altair as alt
import pandas as pd

from calitp_data_analysis import calitp_color_palette as cp
from calitp_data_analysis import styleguide
from segment_speed_utils.project_vars import SEGMENT_GCS

In [2]:
SPEEDS_SHAPE = "speeds_by_shape_peak_daytype"
SPEEDS_ROUTE = "speeds_by_route_direction_peak_daytype"

months = ["apr", "oct"]

df = pd.concat(
    [pd.read_parquet(
            f"{SEGMENT_GCS}rollup/{SPEEDS_SHAPE}_{m}2023.parquet"
        ) for m in months
    ], axis=0, ignore_index=True)

In [3]:
def base_bar_by_time_bin(df: pd.DataFrame, y_col: str) -> alt.Chart:
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x=alt.X("stop_sequence:O", title = "Stop Sequence",
                    axis=alt.Axis(labels=False)),
             y=alt.Y(f"{y_col}:Q", scale = alt.Scale(domain=[0, 45])),
             color=alt.Color(
                 "peak_offpeak:N", 
                scale = alt.Scale(range=cp.CALITP_CATEGORY_BRIGHT_COLORS[2:])
            ),
             tooltip = ["organization_name", "stop_sequence", 
                        "stop_id", y_col]
         ).properties(
             width=350, height=250,
        )#.facet("peak_offpeak:N", columns=1)
        .interactive()
    )
    return chart
    

def chart_by_time_bin(
    df: pd.DataFrame,
    y_col: str,
    shape_id: str
) -> alt.Chart:
    
    subset_df = df[df.shape_id==shape_id]
    
    weekday_df = subset_df[
        subset_df.weekday_weekend=="weekday"].reset_index(drop=True)
    weekend_df = subset_df[
        subset_df.weekday_weekend=="weekend"].reset_index(drop=True)
    
    weekday_chart = (base_bar_by_time_bin(weekday_df, y_col)
                     .properties(title = f"{shape_id} Weekday Speeds by Stop")
                    )
    
    weekend_chart = (base_bar_by_time_bin(weekend_df, y_col)
                     .properties(title = f"{shape_id} Weekend Speeds by Stop")
                    )
 
    chart = alt.vconcat(weekday_chart, weekend_chart)
    
    return chart

In [4]:
operator = "Los Angeles County Metropolitan Transportation Authority"
test_shapes = ["1690148_JUNE23", "900359_DEC22"]
test = df[(df.organization_name==operator) & 
          (df.shape_id.isin(test_shapes))
         ].reset_index(drop=True)

In [5]:
chart_by_time_bin(
    test, 
    "p50_mph",
    test_shapes[0]
)

alt.VConcatChart(...)